## All Headers

In [1]:
import numpy as np
import random
import time
import cirq

## Defining Common Util Functions

In [2]:
'''
Takes a qubitString as input and returns a matrix 
that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a bitstring, return the decimal number 
represented by the bitstring
'''
def getDecimalNo(bitString):
    val = 0
    n = len(bitString)
    for i in range(0,n):
        val = val + (2**i)*(int)(bitString[-i-1])
    return val

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

## Common function object. 
This is the base class and all functions Deutsch-Jozsa (balanced, constant), Bernstein-Vazirani(different a and b values) inhert from this class

In [3]:
class FunctionObject:
    
    def __init__(self,fx,n):
        self.__fx = fx
        self.__n = n
        self.__Uf = self.__createUf()
    
    '''
    Apply the function on the given input.
    '''
    def applyFx(self,input):
        return self.__fx(input)
        
    '''
    The N value which signifies the no. of qubits.
    '''
    def getN(self):
        return self.__n
    
    '''
    The Uf matrix which represents the oracle for this function
    '''
    def getUf(self):
        return self.__Uf
    
    '''
    Given the function to execute, and the qubits |x1x2..xn> and the ancilla|b>.
    This function will return a bit vector representing |x1x2..xn> |b+f(x)>
    '''
    def getFunctionResult(self,x,ancilla):
        fx = self.applyFx(x)
        newAncilla = str((((int)(ancilla) + fx)%2))
        result = x+newAncilla
        return getQubitVector(result)
    
    '''
    Using pointer to f(x) and n denoting the no. of qubits,
    it returns an oracle matrix Uf of size 2^(n+1)x2^(n+1) which
    can be used in a Deutsch-Jozsa or Bernstein-Vazirani circuit
    '''
    def __createUf(self):
        Uf = np.zeros((2**(self.__n+1),2**(self.__n+1)))    
    
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0

        inputs = getAllPossibleNBitStrings(self.__n+1)

        for i in inputs:
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = i[0:self.__n]

            #fx represents the output of function f given the input x
            fx = str(self.applyFx(x))

            #b is the last bit of the bit pattern in the dictionary item
            b = i[self.__n]

            #below we have the (f(x) + b) mod 2
            if(b==fx):
                bfx = '0'
            else:
                bfx = '1'
                # print(bfx)

            #the final bit string is the concatenation of the input x and bfx
            result = x + bfx

            #using indices_dict we can now find the index that corresponds to this output
            column = getDecimalNo(result)
            row = getDecimalNo(i)
            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[row][column] = 1        
        return Uf
    
    '''
    Given a function and n qubits. This function will verify if the Uf matrix 
    generated for this function matches the expected output. 
    i.e., it checks if Uf|x1x2..xn>|b> = |x1x2..xn> |b+f(x)>
    '''
    def verifyUf(self,debug=False):
        
        UfMatrix = self.getUf()
        if debug:
            print("\nUfMatrix for function \n"+str(UfMatrix)+"\n")

        inputs = getAllPossibleNBitStrings(self.__n+1)
        valid = True

        for i in inputs:
            inputBitVector = getQubitVector(i)

            if debug:
                print("Input BitString {}".format(i))
                print("Input BitVector {}\n".format(inputBitVector))

            functionOutput = self.getFunctionResult(i[0:self.__n],i[self.__n])
            UfOutput = np.matmul(UfMatrix,inputBitVector)

            if debug:
                print("\tUf Output {}".format(UfOutput))
                print("\t F Output {}\n".format(functionOutput))

            if np.array_equal(UfOutput,functionOutput) is not True:
                print("ERROR FOR {}\n".format(i))
                valid=False

        return valid

class Oracle (cirq.Gate):
    
    def __init__(self, N, f):
        self.__n = N
        self.functionObj = f
        
    def num_qubits(self):
        
        return self.__n # replace with the number of qubits needed for the operation

    def _unitary_(self):
        
        return self.functionObj.getUf() # replace with a numpy array of the unitary matrix

    def __str__(self):
        return "Uf" # optional, but allows the gate to be named when you print the circuit

## Bernstein Vazirani Function Object
A subclass of the function object, this creates a function object for the Bernstein-Vazirani Algorithm.

In [4]:


class BernsteinVaziraniFunction(FunctionObject):

    '''
    Initializes a function object with the passed in a and b values.
    If no a and b values are passed, 
    it will initialize 'a' to a random n-bit string of 0s and 1s 
    and b to 0 or 1.
    '''
    def __init__(self,n,a=None,b=None):
        if a is None:
            self.initRandomAandBValues(n)  
        else:
            self.__a = a
            self.__b = b
        FunctionObject.__init__(self, self.fx, n) 
        
    '''
    Initializes random values for a and b
    '''
    def initRandomAandBValues(self,n):
        self.__b = random.randint(0,1)
        self.__a = ''
        for i in range(0,n):
            self.__a+=str(random.randint(0,1))
        return
        
    '''
    Applies the function f(x) = a*x + b to the inputString
    '''
    def fx(self,inputString):
        dotProduct = 0
        for i in range(0,len(self.__a)):
            dotProduct+= ((int)(inputString[i]))*((int)(self.__a[i]))
        dotProduct = dotProduct % 2
        return (dotProduct + self.__b) % 2
    
    def getA(self):
        return self.__a
    
    def getB(self):
        return self.__b

## Quantum Circuit
The common quantum circuit used for both Deutsch-Jozsa and Berstein-Vazirani Algorithm.

<img src="circuit.png" alt="Circuit" style="width: 300px; float: left"/>

In [5]:
n=3
qubits = cirq.LineQubit.range(n)
c=cirq.Circuit()
c.append(cirq.X(qubits[n-1]))
for i in range(0,n):
    c.append([cirq.H(qubits[i])])



c.append(cirq.measure(*qubits))
print(c)
simulator = cirq.Simulator()
result = simulator.run(c, repetitions=1)
print(str(result).split('=')[1].split(','))

0: ───H───────M───
              │
1: ───H───────M───
              │
2: ───X───H───M───
['1', ' 0', ' 1']


In [6]:
def runMainCircuit(functionObj,nTrials,debug=False):
    
    c = cirq.Circuit()

    #create a matrix representing Uf
    #UfMatrix = functionObj.getUf()
    
    #for a n bit function, we need n+1 qubits(one ancilla bit)
    n = functionObj.getN()+1
    
    
    UfMatrix = Oracle(n, functionObj)
    
    
    qubits = cirq.LineQubit.range(n)
    
    #setting last qubit to 1
    c.append(cirq.X(qubits[n-1]))
    
    #adding Hadamard gates to all qubits
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])

    #GateName = "UF_GATE"

    #create a gate that uses the Uf matrix and pass all qubits to this as input 
    #uf_gate_definition = DefGate(GateName, UfMatrix)
    #qubits = [unpack_qubit(i) for i in range(0,n)]

    #adding Uf gate
    #p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits))
    
    c.append(UfMatrix(*qubits))
    
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])

    for i in range(0,n):
        c.append(cirq.measure(qubits[i])) 
        
    print(c)
    
    simulator = cirq.Simulator()
    
    result = simulator.run(c, repetitions=1)
    
    return result

## Post-Process Quantum Circuit Results

Process results of measurement of the above defined quantum circuit to gain information

* a and b values in Bernstein-Vazirani

In [7]:
import time

nTrials = 1

    
'''
Given a function f(x)=a.(x)+b and the number of qubits,
this function uses the quantum circuit to identify the values of a and b
'''
def constructBVFunction(functionObj):
    
    b = functionObj.applyFx("0"*functionObj.getN())
    a = ""
    start = time.time()
    results = runMainCircuit(functionObj,nTrials)
    
    end = time.time()
    
    if results is None:
        return None,None,None
    
    timetaken = end-start
    result_arr= str(results).split('=')[1].split(',')
    
    for i in range(0,functionObj.getN()):
        
        a+=result_arr[i]
        
    return a,b,timetaken

## Experiments with Quantum Circuits

In [8]:
'''
Driver that verifies the validity of the quantum circuits for different F configurations.
Here F is randomly initialized as ax+b where a is a bit-string of size n, randomly initialized
and b is randomly initialized to 0 or 1
'''
class BernsteinVaziraniDriver():
    
    def __init__(self,n):
        
        self.functionObj = BernsteinVaziraniFunction(n)
        
    def runAndVerifyQuantumCircuit(self,debug=False):
        
        print("*"*20)
        if debug:
            print("Verified that the created Uf matrix is valid: {}".format(balancedFxN2Object.verifyUf()))

        a,b,circuitRunTime = constructBVFunction(self.functionObj)
        if a is None:
            return
        
        print(a)
        print(self.functionObj.getA())
        assert a == self.functionObj.getA()
        assert b == self.functionObj.getB()
        print("N : {} , A: {}, B: {}, Time : {}".format(self.functionObj.getN(), a,b, circuitRunTime))
        print("*"*20)

In [9]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()

********************
0: ───H───────Uf───H───M───
              │
1: ───X───H───#2───H───M───
0
1
0


AssertionError: 

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()